In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'e:\\EndToEnd\\01_MLOps'

## Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    train_data_dir: Path
    test_data_dir: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_col: str

## configuration

In [21]:
from mlops_wine_quality.utils.common import read_yaml, create_dirs
from mlops_wine_quality.constants import *

class ConfigManger:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        target_col = self.schema.TARGET_COLUMN
        params = self.params.Elasticnet

        create_dirs([config.root_dir])

        return ModelTrainingConfig(
            root_dir= config.root_dir,
            train_data_dir= config.train_data_dir,
            test_data_dir= config.test_data_dir,
            model_name= config.model_name,
            alpha= params.alpha,
            l1_ratio= params.l1_ratio,
            target_col= target_col.name
            
        )


## Components

In [22]:
# import elasticnet_regression
from sklearn.linear_model import ElasticNet
import joblib
import pandas as pd
from mlops_wine_quality import logger

class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
    
    def start_tarining(self):
        train_data = pd.read_csv(self.config.train_data_dir)
        test_data = pd.read_csv(self.config.test_data_dir)

        x_train = train_data.drop(self.config.target_col, axis=1)
        y_train = train_data[self.config.target_col]
        x_test = test_data.drop(self.config.target_col, axis=1) 
        y_test = test_data[self.config.target_col]

        lr_model = ElasticNet(
            alpha=self.config.alpha,
            l1_ratio=self.config.l1_ratio,
            random_state=42
        )
        lr_model.fit(x_train, y_train)
        # svae the model
        joblib.dump(lr_model, os.path.join(self.config.root_dir, self.config.model_name))



In [23]:
try:
    config = ConfigManger().get_model_training_config()
    logger.info("Training started")
    training = ModelTraining(config)
    training.start_tarining()
    logger.info("Training completed")
except Exception as e:
    raise e

[2024-01-21 00:08:42,128: INFO: common: Yaml file loaded successfully]
[2024-01-21 00:08:42,135: INFO: common: Yaml file loaded successfully]
[2024-01-21 00:08:42,137: INFO: common: Yaml file loaded successfully]
[2024-01-21 00:08:42,139: INFO: common: Created directory at artifacts/model_training]
[2024-01-21 00:08:42,140: INFO: 2605612774: Training started]
[2024-01-21 00:08:42,176: INFO: 2605612774: Training completed]
